In [ ]:
#合并 chinese-alpaca-plus-lora-7b
!cd Chinese-LLaMA-Alpaca && python scripts/merge_llama_with_chinese_lora.py \
    --base_model /home/llama/llama-dl/llama_7b_hf \
    --lora_model /home/llama/chinese-llama-plus-lora-7b,/home/llama/chinese-alpaca-plus-lora-7b \
    --output_type pth \
    --output_dir /home/llama/merge_alpaca_plus_lora_7b_pt

#验证SHA256值  Chinese-Alpaca-Plus-7B 8b8f6551d0d83f93e378622b9f8dad0bec189da6c29d8a78de493e6aee9bd35f
!sha256sum /home/merge_alpaca_plus_lora_7b_pt/consolidated.00.pth

In [ ]:
!mkdir Chinese-LLaMA-Alpaca/pt_data
!cp Chinese-LLaMA-Alpaca/data/pt_sample_data.txt Chinese-LLaMA-Alpaca/pt_data

In [ ]:
#创建基于原模型扩充词表
!cd Chinese-LLaMA-Alpaca/scripts/merge_tokenizer/ && python merge_tokenizers.py \
  --llama_tokenizer_dir /home/llama/llama-dl/llama_7b_hf \
  --chinese_sp_model_file /home/workspace/Chinese-LLaMA-Alpaca/scripts/merge_tokenizer/chinese_sp.model

#训练模型
!cd Chinese-LLaMA-Alpaca/scripts/training && torchrun --nnodes 1 --nproc_per_node 1 run_clm_pt_with_peft.py \
    --deepspeed ds_zero2_no_offload.json \
    --model_name_or_path /home/llama/llama-dl/llama_7b_hf \
    --tokenizer_name_or_path /home/workspace/Chinese-LLaMA-Alpaca/scripts/merge_tokenizer/merged_tokenizer_hf \
    --dataset_dir /home/workspace/Chinese-LLaMA-Alpaca/pt_data \
    --data_cache_dir data_cache \
    --validation_split_percentage 0.001 \
    --per_device_train_batch_size 1 \
    --do_train \
    --fp16 \
    --seed $RANDOM \
    --max_steps 100 \
    --lr_scheduler_type cosine \
    --learning_rate 2e-4 \
    --warmup_ratio 0.05 \
    --weight_decay 0.01 \
    --logging_strategy steps \
    --logging_steps 10 \
    --save_strategy steps \
    --save_total_limit 3 \
    --save_steps 50 \
    --gradient_accumulation_steps 1 \
    --preprocessing_num_workers 8 \
    --block_size 512 \
    --output_dir /home/workspace/Chinese-LLaMA-Alpaca/output_model \
    --overwrite_output_dir \
    --ddp_timeout 30000 \
    --logging_first_step True \
    --lora_rank 8 \
    --lora_alpha 32\
    --trainable q_proj,v_proj,k_proj,o_proj,gate_proj,down_proj,up_proj \
    --modules_to_save embed_tokens,lm_head \
    --lora_dropout 0.05 \
    --torch_dtype float16 \
    --gradient_checkpointing \
    --ddp_find_unused_parameters False

#或使用脚本训练模型
!cd Chinese-LLaMA-Alpaca/scripts/training && ./run_pt.sh

#合并模型
!cd Chinese-LLaMA-Alpaca && python scripts/merge_llama_with_chinese_lora.py \
    --base_model /home/llama/llama-dl/llama_7b_hf \
    --lora_model /home/workspace/Chinese-LLaMA-Alpaca/output_model/pt_lora_model \
    --output_type huggingface \
    --output_dir /home/workspace/Chinese-LLaMA-Alpaca/output_model/hf_lora_model

#启动原训练合并后的hf模型演示
!python scripts/inference/inference_hf.py --base_model /home/workspace/Chinese-LLaMA-Alpaca/output_model/hf_lora_model --with_prompt --interactive --gpus 0

In [ ]:
#启动模型demo
!python scripts/inference/inference_hf.py --base_model /home/llama/merge_alpaca_plus_lora_7b_hf --with_prompt --interactive --gpus 0

#启动web demo
!python scripts/inference/gradio_demo.py --base_model /home/llama/merge_alpaca_plus_lora_7b_hf --gpus 0